In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import matplotlib.pyplot as plt
import numpy as np

In [2]:
device = torch.device("cude" if torch.cuda.is_available() else "cpu")

In [3]:

class CNN(nn.Module):
    def __init__(self, in_channel=1, num_classes=10):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=8, kernel_size=(3,3), stride=(1,1) ,padding=(1,1) )
        self.pool = nn.MaxPool2d(kernel_size= (2,2), stride = (2,2))
        self.conv2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=(3,3), stride=(1,1) ,padding=(1,1) )
        self.fc1 = nn.Linear(16*7*7, num_classes)

    def forward(self,x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fc1(x)

        return x



In [4]:
# Hyperparameters

in_channel = 1 # mnist
num_classes = 10
num_epochs = 5
batch_size = 64
learning_rate=0.01

In [5]:
train_dataset = datasets.MNIST(root="dataset_mnist/", train=True, transform=transforms.ToTensor(), download = True)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_dataset = datasets.MNIST(root="dataset_mnist/", train=False, transform=transforms.ToTensor(), download=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

In [6]:
import os
print("Current Dir :", os.getcwd())
print(os.listdir(os.path.join(os.getcwd(),"dataset_mnist")))

Current Dir : /home/avishkar/Desktop/projects/deep_learning_essentials
['MNIST']


In [7]:
model = CNN()
# x = torch.randn(64, 1, 28, 28)
# print(model(x).shape)
print("Model parameters :", sum(p.numel() for p in model.parameters()))

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

Model parameters : 9098


In [ ]:
for epoch in range(num_epochs):
    for batch_idx, (data, targets) in enumerate(train_loader):
        data = data.to(device)
        targets = targets.to(device)

        # reshape, flatten ?
        # print(data.shape)
        # print(data[0].shape)
        # data = data.reshape(data.shape[0], -1)
        # print(data.shape)

        #forward
        scores = model(data)
        loss = criterion(scores, targets)
        # backward
        optimizer.zero_grad()
        loss.backward()

        # gradient descent
        optimizer.step()
    print(f"Epoch {epoch} loss : {loss}")


KeyboardInterrupt: 

In [ ]:
def check_accuracy(loader, model):
    if loader.dataset.train:
        print("checking traning data accuracy")
    else:
        print("print testing data acc")

    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x,y in loader:
            x = x.to(device)
            y = y.to(device)
            # x = x.reshape(x.shape[0], -1)

            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

        print(f"Got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}")

    model.train()

In [ ]:
check_accuracy(train_loader, model)
check_accuracy(test_loader, model)

In [ ]:
# fully connected NN
class NN(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x



In [ ]:
model = NN(input_size=784, hidden_size=500, num_classes=10)
print(model)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)


NN(
  (fc1): Linear(in_features=784, out_features=500, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=500, out_features=10, bias=True)
)


In [ ]:
total_steps = len(train_loader)
num_epochs = 5
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.reshape(-1, 28*28)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_steps, loss.item()))

Epoch [1/5], Step [100/938], Loss: 2.0351
Epoch [1/5], Step [200/938], Loss: 1.7629
Epoch [1/5], Step [300/938], Loss: 1.3962
Epoch [1/5], Step [400/938], Loss: 1.1116
Epoch [1/5], Step [500/938], Loss: 0.8915
Epoch [1/5], Step [600/938], Loss: 0.8219
Epoch [1/5], Step [700/938], Loss: 0.6315
Epoch [1/5], Step [800/938], Loss: 0.6969
Epoch [1/5], Step [900/938], Loss: 0.6686
Epoch [2/5], Step [100/938], Loss: 0.6812
Epoch [2/5], Step [200/938], Loss: 0.6842
Epoch [2/5], Step [300/938], Loss: 0.4446
Epoch [2/5], Step [400/938], Loss: 0.4970
Epoch [2/5], Step [500/938], Loss: 0.5159
Epoch [2/5], Step [600/938], Loss: 0.3292
Epoch [2/5], Step [700/938], Loss: 0.5313
Epoch [2/5], Step [800/938], Loss: 0.4760
Epoch [2/5], Step [900/938], Loss: 0.3715
Epoch [3/5], Step [100/938], Loss: 0.4512
Epoch [3/5], Step [200/938], Loss: 0.5021
Epoch [3/5], Step [300/938], Loss: 0.4730
Epoch [3/5], Step [400/938], Loss: 0.2973
Epoch [3/5], Step [500/938], Loss: 0.4545
Epoch [3/5], Step [600/938], Loss:

In [ ]:
# Testing the model
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, 28*28)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))

Accuracy of the network on the 10000 test images: 91.75 %
